In [ ]:
# parametres
time_rotation = 2.5
speed_r = 125
speed_f = 100

periode = 1

In [ ]:
#input : path, table of the points to reach 
#return : teta, table of the angles taken by the robot to reach each point
def diff_angle (path):
    i = 1
    while i < (np.shape(path)[0]):
        teta [i] = math.atan((path[i,0] - path[i-1,0])/(path[i,1] - path[i-1,1]))
        i = i + 1

In [ ]:
#input : angle to change the trajectory
def orientation (angle):
    if angle < 0:
        motor_left_target = speed_r
        motor_right_target = -speed_r
    else :
        motor_left_target = -speed_r
        motor_right_target = speed_r
        
    time.sleep(time_rotation * angle / math.pi)
    stopmotors()

In [ ]:
#input : coordinates of two different points 
#return the distance between two points 
def distance (x1, y1, x2, y2):
    dist = sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

In [ ]:
def mouving (path):
    teta = diff_angle(path)
    
    goal = path[0]
    i = 0
    x_pense = 0
    y_pense = 0
    teta_pense = 0

    while i < np.shape(path)[0]:
        [deltax, delaty, deltateta] = kalman()
        teta[i] = math.atan((goal[i,0] - (x_pense + deltax))/(goal[i,1] - (y_pense + deltay)))
        orientation(teta[i] - (teta_pense + deltateta)
        dist = distance (goal[i,0], goal[i,1], x_pense + deltax, y_pense + deltay)
        if dist < (speed_f * periode) :
            i = i + 1
            goal = path[i]
            motor_left_target = speed_f
            motor_right_target = speed_f
            time.sleep(distance/(speed_f/25))
            orientation(teta[i] - teta[i-1])
        else :
            motor_left_target = speed_f
            motor_right_target = speed_f
            x_pense = x_pense + deltax + speed_f * periode * math.cos(teta[i])
            y_pense = y_pense + deltay + speed_f * periode * math.sin(teta[i])
            teta_pense = teta[i]
            time.sleep(periode)
